## Imports
These experiments were run on Python 3.8. In the requirements.txt are the versions used for these packages.
- tqdm: For showing progress in loops.
- numpy and pandas: For data manipulation.
- cornac: For obtaining the recommendations.
- tensorflow: Required by cornac.
- torch: Required for the VAECF implementation of Cornac.

In [ ]:
from datetime import datetime
from pathlib import Path
from logging import Formatter, StreamHandler, getLogger, INFO

from tqdm import tqdm
from cornac import Experiment
from cornac.eval_methods import RatioSplit
from cornac.metrics import NDCG, Recall, Precision
from cornac.models import MF, WMF, SVD, VAECF
from cornac.exception import ScoreException
from numpy import array, nan
from pandas import read_csv, DataFrame, Series

## Logger setup
Here we set up the logger for showing some info when executing this script.

In [ ]:
logger = getLogger(__name__)
logger.setLevel(INFO)

ch = StreamHandler()
ch.setLevel(INFO)
ch.setFormatter(
    Formatter('%(asctime)s - %(levelname)s - %(message)s')
)

logger.addHandler(ch)

## Configuration variables
We define some variables used on the rest of the experiment.

### General config
Getting the date now and the name of the experiment.

In [ ]:
now = f'{datetime.now():%Y%m%d%H%M%S}'
experiment_name = 'AMBAR'

### File and dir config
Getting the working directory with pathlib, and obtaining the csv to be used in cornac, and defining a results directory.

In [ ]:
work_dir = Path('.').resolve()
data_file = work_dir / 'data' / experiment_name / 'ratings_info.csv'
results_dir = work_dir / 'results' / 'AMBAR' / experiment_name / now

Here we make sure the results directory exists by creating it if it doesn't.

In [ ]:
if not results_dir.exists():
    results_dir.mkdir(parents=True, exist_ok=True)

Also, we make sure the data file exists and is a file. Here we could also make sure that the file is an actual csv file.

In [ ]:
if not data_file.exists() and data_file.is_file():
    print("Bad data file")

### Dataframe config
We define the names of the headers of each column to be identified by pandas. Also, we define the data type of the values in each cell of the user, item and rating. If the data has multiple data types, the val_dtype can be a list of type string compatible with pandas.

In [ ]:
col_names = {
    'user': 'user_id',
    'item': 'track_id',
    'rating': 'rating'
}
val_dtype = 'int'

### Cornac config
Here we set up the k value, the test set size and the validation set size. Also we decide if we want to exclude unknown values or not.

In [ ]:
k = 1000
test_size = 0.2
val_size = 0.1
exclude_unknown = True

## Function setup
We set up various utility functions to be used later. Mostly for exporting data and getting it in a format compatible with cornac.

set_data_to_tuple_list takes a dict of {user: [item_list, rating_list]}, process it and returns a tuple list of format [(user, item, rating)...].

In [ ]:
def set_data_to_tuple_list(d: dict) -> list:
    result = []
    for user in d:
        transpose = array(d[user]).T
        for t in transpose:
            result.append((user,) + tuple(t))
    return result

list_to_dict converts a list into a dict using dict comprehension and enumerate.

In [ ]:
def list_to_dict(l: list) -> dict:
    return {i: v for i, v in enumerate(l)}

get_set_dataframe process the raw data ({user: [item_list, rating_list]}), with the item ids and user ids, and converts it into a pandas DataFrame to be exported later.

In [ ]:
def get_set_dataframe(set_data: dict, i_ids: list, u_ids: list) -> DataFrame:
    data_list = set_data_to_tuple_list(set_data)
    i_map = list_to_dict(i_ids)
    u_map = list_to_dict(u_ids)

    set_df = DataFrame(data_list,
                       columns=list(col_names.values()),
                       dtype=val_dtype)
    set_df['item_idx'] = set_df[col_names['item']]
    set_df['item'] = set_df[col_names['item']].replace(to_replace=i_map)
    set_df['user_idx'] = set_df[col_names['user']]
    set_df['user'] = set_df[col_names['user']].replace(to_replace=u_map)
    return set_df

In [ ]:
logger.info('Experiment start...')
logger.info(f'{experiment_name}')
logger.info(f'{k=}')
logger.info(f'{work_dir=}')
logger.info(f'{data_file=}')
logger.info(f'{results_dir=}')

Here we create the dataset out of the data file, the expected data is only with user, item and rating in that order. The name of the columns is defined in the set-up part, same with the data types.

For testing purposes before actually executing the full experiment, we left a filter that takes a sample of 50 users, and gets only the data of those 50 users. Please use it only to make sure that the script executes correctly from start to finish.

In [ ]:
keys = ['0', '1', '2']

if isinstance(val_dtype, str):
    d_type = {key: val_dtype for key in keys}
elif isinstance(val_dtype, list):
    d_type = dict(zip(keys, val_dtype))
else:
    logger.error('Wrong type setup. Must be a type string or a list of type string.')
    exit()

logger.info('Loading data into triplets...')
df = read_csv(
    data_file,
    header=0,
    names=['0', '1', '2']
)[['0', '1', '2']].astype(d_type)

# FOR TESTING ONLY
# user_filter = Series(df['0'].unique()).sample(50).to_list()
# df = df[df['0'].isin(user_filter)]

data = list(df.to_records(index=False, column_dtypes=d_type))

Here we create the Ratio Split that will be used by cornac. It splits the data into 3 sets randomly. 1 for test, 1 for train and 1 for validation.

In [ ]:
logger.info('Creating ratio split...')
ratio_split = RatioSplit(
    data=data,
    test_size=test_size,
    val_size=val_size,
    exclude_unknowns=exclude_unknown,
    verbose=True
)

We define the metris here. In this experiment, we set up NDCG, Recall and Precision, using the k defined in the set-up.

In [ ]:
metrics = [
    NDCG(k),
    Recall(k),
    Precision(k)
]

Also, we define the models with some previously obtained parameters. We could also define the hyperparameter calculation in this part, in this case, is important to leave a models variable with said configuration, so cornac can pick up the array and execute the calculation and exporting of the recommendations.

Because this script is assuming an array with models with parameters already predefined, in case of needing the best parameters obtained by cornac, the exporting of this must be done after running the experiment.

In [ ]:
models = [
    MF(
        max_iter=5,
        k=k,
        early_stop=True,
        verbose=True,
        lambda_reg=0.001,
        learning_rate=0.01,
        use_bias=False
    ),
    WMF(
        k=k,
        max_iter=50,
        learning_rate=0.001,
        lambda_u=0.01,
        lambda_v=0.01,
        verbose=True,
    ),
    SVD(
        max_iter=5,
        k=k,
        early_stop=True,
        verbose=True,
        lambda_reg=0.0001,
        learning_rate=0.0001
    ),
    VAECF(
        k=k,
        autoencoder_structure=[20],
        act_fn="tanh",
        likelihood="mult",
        n_epochs=100,
        batch_size=100,
        learning_rate=0.001,
        beta=1.0,
        use_gpu=True,
        verbose=True
    ),
]

In [ ]:
total_users = ratio_split.train_set.num_users
total_items = ratio_split.train_set.num_items
logger.info(f'{total_users=}')
logger.info(f'{total_items=}')

After setting up the metrics and models, we export the test, train and validation data into the results directory.

In [ ]:
logger.info('Exporting test data...')
get_set_dataframe(
    dict(ratio_split.test_set.user_data),
    list(ratio_split.test_set.item_ids),
    list(ratio_split.test_set.user_ids),
).to_csv(results_dir / 'test_set.csv')

In [ ]:
logger.info('Exporting train data...')
get_set_dataframe(
    dict(ratio_split.train_set.user_data),
    list(ratio_split.train_set.item_ids),
    list(ratio_split.train_set.user_ids),
).to_csv(results_dir / 'train_set.csv')

In [ ]:
logger.info('Exporting validation data...')
get_set_dataframe(
    dict(ratio_split.val_set.user_data),
    list(ratio_split.val_set.item_ids),
    list(ratio_split.val_set.user_ids),
).to_csv(results_dir / 'val_set.csv')

And we run the experiments with the defined variables.

In [ ]:
logger.info('Running experiment...')
exp = Experiment(
    eval_method=ratio_split,
    models=models,
    metrics=metrics,
    user_based=True,
)
exp.run()

After running the experiment, we export the metrics obtained from the calculation into a csv using pandas.

In [ ]:
logger.info('Exporting metrics...')
metric_results = {
    exp.models[i].name: dict(exp.result[i].metric_avg_results)
    for i in range(len(models))
}
(DataFrame(metric_results)
 .reset_index()
 .rename(columns={'index': 'metric'})
 .to_csv(results_dir / 'metric_results.csv'))

And finally we export the recommendations. We use a custom multi loop to get the results.
- Here we first loop over the models of the experiment.
- We loop over the users map of cornac to get both the original id and the internal index of cornac.
- We get the scores for the users.
- We get the k top items using a combination of argsort and reversing of the list.
- We loop over the items map of cornac to get both the original id and the internal index of cornac.
- We get the score obtained from cornac, or nan in case of IndexError.
- We append the user and items, both the id and indexes, and the score to the result list.
- After all the loops are finished, we export the data into a csv file.

In [ ]:
logger.info('Processing models...')
for model in exp.models:
    model_result = []
    logger.info(f'Getting scores for {model.name}...')

    for user_id, user_index in tqdm(exp.eval_method.train_set.uid_map.items()):
        try:
            scores = model.score(user_index)
        except ScoreException:
            logger.error(f"{model.name}: Couldn't predict for user {user_index} ({user_id=})")
            continue

        top_items = list(reversed(scores.argsort()))[:k]

        for item_id, item_index in exp.eval_method.train_set.iid_map.items():
            if item_index not in top_items:
                continue

            try:
                score = scores[item_index]
            except IndexError:
                logger.error(
                    f"{model.name}: No score for item {item_index} ({item_id=}) in user {user_index} ({user_id=})"
                )
                score = nan

            model_result.append({
                'user_id': user_id,
                'user_index': user_index,
                'item_id': item_id,
                'item_index': item_index,
                'score': score
            })

    logger.info(f'Exporting {model.name}...')
    (DataFrame(model_result)
     .sort_values(by=['user_id', 'score'], ascending=[True, False])
     .to_csv(results_dir / f'{model.name}.csv'))